# 네이버 기사 카테고리별 크롤링

### 1. 패키지 import

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

### 2. 카테고리 확인

In [2]:
categorys = {
    100:"정치",
    101:"경제",
    102:"사회",
    103:"생화/문화",
    104:"세계",
    105:"IT/과학",
}

### 3. 카테고리, 페이지 별 URL 만들기

In [3]:
def make_url(category, page):
    return "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1={}#&date=%2000:00:00&page={}".format(category, page)

In [4]:
# 정치 2페이지
url = make_url(100, 2)   
url

'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100#&date=%2000:00:00&page=2'

### 4. 기사 리스트 페이지에서 기사의 제목과 링크를 리턴

In [5]:
# url 입력하면 제목과 링크 데이터들을 리턴
def title_link(url):
    
    result = {"title": [], "link": []}
    
    options = webdriver.ChromeOptions()    
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    articles = driver.find_elements_by_css_selector('#section_body li')

    for article in articles:
        title = article.find_element_by_css_selector('dt:not(.photo) > a').text
        link = article.find_element_by_css_selector('dt:not(.photo) > a').get_attribute("href")
        result["title"].append(title)
        result["link"].append(link)
    
    driver.quit()
    df = pd.DataFrame(result)
    df["category"] = category
    return df

- 한페이지에 20개의 리스트가 있음

In [6]:
category, page = 100, 2
url = make_url(category, page)   
print(url)
df = title_link(url)
df.tail(2)

https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100#&date=%2000:00:00&page=2


Empty DataFrame
Columns: [title, link, category]
Index: []

### 5. 상세 페이지 url 입력하면 기사 내용을 크롤링해서 데이터 프레임에 컬럼으로 추가

In [7]:
# 상세 페이지 url 입력하면 기사 내용을 크롤링
def article(url):
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    content = dom.select_one("#articleBodyContents")
    return content.text.replace("\n", "").replace("\'","").split("{}")[1].strip()

In [8]:
# 데이터 프레임에 기사 데이터 추가
contents = []
for link in df["link"]:
    contents.append(article(link))
df["content"] = contents
df.tail()

Empty DataFrame
Columns: [title, link, category, content]
Index: []

### 6. 함수로 만들기
- 카테고리와 페이지 입력하면 제목, 기사, 카테고리가 있는 데이터 프레임 생성

In [9]:
def get_article(category, page):
    
    # url 만들기
    url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1={}#&date=%2000:00:00&page={}".format(category, page)
    
    # 제목, 링크 리스트 가져오기
    result = {"title": [], "link": []}
    
    options = webdriver.ChromeOptions()    
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)

    driver.get(url)
    articles = driver.find_elements_by_css_selector('#section_body li')

    for article in articles:
        title = article.find_element_by_css_selector('dt:not(.photo) > a').text
        link = article.find_element_by_css_selector('dt:not(.photo) > a').get_attribute("href")
        result["title"].append(title)
        result["link"].append(link)

    driver.quit()
    
    df = pd.DataFrame(result)
    df["category"] = category
    
    # 기사 내용 크롤링 하기
    contents = []
    for link in df["link"]:
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one("#articleBodyContents")
        content = content.text.replace("\n", "").replace("\'","").split("{}")[1].strip()
        contents.append(content)
    df["content"] = contents
    return df

In [10]:
%%time
result = pd.DataFrame(columns=["title", "link", "category", "article"])
for category in categorys.keys():
    for page in range(1, 3):
        print(category, page)
        df = get_article(category, page)
        result = pd.concat([result, df], sort=False)

100 1
100 2
101 1
101 2
102 1
102 2
103 1
103 2
104 1
104 2
105 1
105 2
CPU times: user 195 ms, sys: 100 ms, total: 295 ms
Wall time: 6min 17s


In [11]:
result.reset_index(drop=True, inplace=True)
result.tail()

Empty DataFrame
Columns: [title, link, category, article, content]
Index: []

In [12]:
result.dtypes

title        object
link         object
category     object
article      object
content     float64
dtype: object

In [13]:
result.dtypes

title        object
link         object
category     object
article      object
content     float64
dtype: object

In [14]:
# save
result.to_csv("article.csv", index=False)